In [22]:
from datasets import Dataset, load_dataset

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import v2

In [21]:
# Hyper-parameters

learning_rate = 1e-3
batch_size = 64

In [10]:
# Import data
dataset = load_dataset("camfruss/bread_proofing")

train_ds = dataset["train"]
valid_ds = dataset["valid"]
test_ds = dataset["test"]

In [23]:
# modify so labels in correct format [0, 0, 0, 0]
test_ds

Dataset({
    features: ['image', 'upvotes', 'under_proof', 'over_proof', 'perfect_proof', 'unsure_proof'],
    num_rows: 381
})

In [ ]:
dataset = Dataset.from_dict({"data": data, "label": label}).with_format("torch")
train_dl = DataLoader(dataset, batch_size=batch_size)

dataset = Dataset.from_dict({"data": data, "label": label}).with_format("torch")
valid_dl = DataLoader(dataset, batch_size=batch_size)

dataset = Dataset.from_dict({"data": data, "label": label}).with_format("torch")
test_dl = DataLoader(dataset, batch_size=batch_size)

del dataset
del train_ds
del valid_ds
del test_ds

In [17]:
dim = 2**7
out_channels = 16
fc_features = out_channels * dim // 4

class ConvolutionalNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            v2.Grayscale(),
            v2.Resize((dim, dim)),
            
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, stride=1),  # default padding of 0s
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # halves
            
            nn.Conv2d(in_channels=8, out_channels=out_channels, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # halves
            
            nn.Flatten(),
            
            nn.Linear(in_features=fc_features, out_features=fc_features // 4),
            nn.ReLU(),
            
            nn.Linear(in_features=fc_features // 4, out_features=4),
            nn.Softmax()
        )

    def forward(self, x):
        return self.cnn(x)

In [19]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = ConvolutionalNN().to(device)

ConvolutionalNN(
  (cnn): Sequential(
    (0): Grayscale(num_output_channels=1)
    (1): Resize(size=[128, 128], interpolation=InterpolationMode.BILINEAR, antialias=True)
    (2): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=512, out_features=128, bias=True)
    (10): ReLU()
    (11): Linear(in_features=128, out_features=4, bias=True)
    (12): Softmax(dim=None)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
def train(dataloader):
    model.train()
    
    size = len(dataloader.dataset)
    for idx, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
    
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        
        if idx % 100 == 0:
            loss, current = loss.item(), idx * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, split="Validation"):
    model.eval()
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"{split} Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")     

In [ ]:
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n{20*'-'}")
    train(train_dl)
    test(valid_dl)

print(f"Completed {epochs} epochs!")

test(test_dl, split="Test")